In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
df = {
    'Fund Name': ['M1', 'M2', 'M3', 'M4', 'M5','M6','M7','M8'],
    'P1': [0.84, 0.91, 0.79, 0.78, 0.94, 0.88, 0.66, 0.93],
    'P2': [0.71, 0.83, 0.62, 0.61, 0.88, 0.77, 0.44, 0.86],
    'P3': [6.7, 7, 4.8, 6.4, 3.6, 6.5, 5.3, 3.4],
    'P4': [42.1, 31.7, 46.7, 42.4, 62.2, 51.5, 48.9, 37],
    'P5': [12.59, 10.11, 13.23, 12.55, 16.91, 14.91, 13.83, 10.55]
}

d = pd.DataFrame(df)

In [4]:
d

,Fund Name,P1,P2,P3,P4,P5
0,M1,0.84,0.71,6.7,42.1,12.59
1,M2,0.91,0.83,7.0,31.7,10.11
2,M3,0.79,0.62,4.8,46.7,13.23
3,M4,0.78,0.61,6.4,42.4,12.55
4,M5,0.94,0.88,3.6,62.2,16.91
5,M6,0.88,0.77,6.5,51.5,14.91
6,M7,0.66,0.44,5.3,48.9,13.83
7,M8,0.93,0.86,3.4,37.0,10.55


In [5]:
mm = d.shape[0]
mm

8

In [6]:
w = [1,1,0.5,2,1]
im = ['-','+','+','+','-']

In [7]:
Dataf = d
Name = d[d.columns[0]]
D = d.drop(d.columns[0],axis=1)

In [8]:
#Step 1: convert categorical to numerical
def check_encode(D):
    for column in D.columns:
        if pd.to_numeric(D[column]).all():
            continue
        else:
            le = LabelEncoder()
            D[column] = le.fit_transform(D[column])
    return D

D=check_encode(D)


In [9]:
#Step 2: vector normalization
n = len(D.columns)
print(n)
colSqSum = (D**2).sum()
colSqSumarray = colSqSum.values
print(colSqSumarray)
colSqSumRoot = np.sqrt(colSqSumarray)
print(colSqSumRoot)
vn = D.div(colSqSumRoot)
print(vn)

5
[5.7247000e+00 4.2500000e+00 2.5275000e+02 1.7037250e+04 1.4040832e+03]
[  2.39263453   2.06155281  15.8981131  130.52681717  37.4710982 ]
         P1        P2        P3        P4        P5
0  0.351077  0.344401  0.421434  0.322539  0.335992
1  0.380334  0.402609  0.440304  0.242862  0.269808
2  0.330180  0.300744  0.301923  0.357781  0.353072
3  0.326000  0.295893  0.402563  0.324837  0.334925
4  0.392872  0.426863  0.226442  0.476530  0.451281
5  0.367795  0.373505  0.408854  0.394555  0.397907
6  0.275847  0.213431  0.333373  0.374636  0.369084
7  0.388693  0.417161  0.213862  0.283467  0.281550


In [10]:
#Step 3: weight assignment
wnv=vn*w
wnv

,P1,P2,P3,P4,P5
0,0.351077,0.344401,0.210717,0.645078,0.335992
1,0.380334,0.402609,0.220152,0.485724,0.269808
2,0.330180,0.300744,0.150961,0.715562,0.353072
3,0.326000,0.295893,0.201282,0.649675,0.334925
4,0.392872,0.426863,0.113221,0.953061,0.451281
5,0.367795,0.373505,0.204427,0.789110,0.397907
6,0.275847,0.213431,0.166686,0.749271,0.369084
7,0.388693,0.417161,0.106931,0.566933,0.281550


In [11]:
#step 4: calculate ideal best and worst values
ib = {}
iw = {}
for i in range(0,n):
    if im[i]=='+':
        ib[i] = wnv[wnv.columns[i]].max()
        iw[i] = wnv[wnv.columns[i]].min()
    else:
        ib[i] = wnv[wnv.columns[i]].min()
        iw[i] = wnv[wnv.columns[i]].max()
print(ib)
print(iw)

{0: 0.27584655800427665, 1: 0.42686270006394605, 2: 0.22015191230285364, 3: 0.9530608552257642, 4: 0.2698079449705219}
{0: 0.3928723704909395, 1: 0.21343135003197303, 2: 0.10693092883281462, 3: 0.4857239406857994, 4: 0.45128114237898376}


In [12]:
wnv=wnv.to_numpy()

In [13]:
#step 5: calculate euclidean distance
distBest = np.zeros(mm)
distWorst = np.zeros(mm)
distB = np.copy(wnv)
distW = np.copy(wnv)

for j in range(mm):
    for i in range(n):
        distB[j][i] = (wnv[j][i] - ib[i]) ** 2
        distW[j][i] = (wnv[j][i] - iw[i]) ** 2
        distWorst[j] += distW[j][i]
        distBest[j] += distB[j][i]

for j in range(mm):
    distWorst[j] = distWorst[j] ** 0.5
    distBest[j] = distBest[j] ** 0.5

print(distBest)
print(distWorst)


[0.33433876 0.47948893 0.29493064 0.3410391  0.24096013 0.23417545
 0.31590742 0.41818628]
[0.26145088 0.2858268  0.27561382 0.24615575 0.5138057  0.36145495
 0.30574381 0.27735692]


In [14]:
#Step 6: performance score
score = []
for i in range(len(distBest)):
    score.append(distWorst[i] / (distBest[i] + distWorst[i]))
print(score)


[0.4388308529017517, 0.37347566800635623, 0.48307159742078826, 0.41920624330185996, 0.6807484886215399, 0.6068443667808436, 0.49182530821214376, 0.39876303594798523]


In [15]:
#step7: rank
pData = pd.DataFrame(data ={'Items':Name , 'Performance':score})
pData['Rank'] = pData['Performance'].rank(ascending=False)

In [16]:
pData

,Items,Performance,Rank
0,M1,0.438831,5.0
1,M2,0.373476,8.0
2,M3,0.483072,4.0
3,M4,0.419206,6.0
4,M5,0.680748,1.0
5,M6,0.606844,2.0
6,M7,0.491825,3.0
7,M8,0.398763,7.0
